In [149]:
import pandas as pd
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('seaborn-v0_8-bright')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline
import plotly.express as px

# Futures Contracts

<specifications>

A *futures contract* price of an asset at time $T$ is $S(T)$ is given by the formula:

\begin{align}
\mathbb{E}[S(T) \mid F_{t}]
\end{align}

A long position in a futures contract is an agreement to receive a *cash flow* (either postive or negative) from changes inthe futures price during the time the position is held. A short position receives the opposite cashflow. 

<u>Specifications</u>

**The Asset**

When the asset is a commodity, there may be quite a variation in the quality of what is available in the market place. The asset's grade must be specified. The *Intercontinental Exchange* (ICE) has specified the asset in its orange juice futures contract that are US Grade A with Brix value not less than 62.5 degrees. For some commodities a range of grades can be delivered with higher grades receiving a premium to lower grades.

Financial assets in futures contracts are generally well defined and unambiguous. There is no need to specify the grade of S&P 500 futures contract. For treasury futures, there are some interesting features of the Treasury bond and Treasury note futures contrat on the Chicago Board of Trade. The underlying asset in the Treasury bond contract is any treasury bond that has a maturity between 15 and 25 years. In the Treasury note futures contract, the maturity can be between 6.5 and 10 years. In both cases, the exchange has a formula for adjusting the price received according to the coupon and maturity of the bond delivered. 

**The Contract Size**

- The contract size specifies the amount of the asset that has to be delivered under one contract. The correct size of the contract depends on the likely user. Whereas the value of agricultural futures contracts might be $\$10,000 \ \text{to} \$20,000$, it is *much higher* for financial futures. For treasury bond futures contracts, the notional size is $\$100,000$.

**Delivery Months**

- A futures contrac is referred to by its delivery month. The exchange must specifiy the precise period during the month when the delivery can be made. For many futures contracts, the delivery period can be the whole month. 

| Calendar Month | January | February | March | April | May | June | July | August | September | October | November | December |
| --------------- | ------- | -------- | ----- | ----- | --- | ---- | ---- | ------ | --------- | ------- | -------- | -------- |
| Expiration Code | F       | G        | H     | J     | K   | M    | N    | Q      | U         | V      | X        | Z        |

- The exchange specifies the first day trading in a particular month's contract can begin and similarly specifies the last day trading may take place for a given contract. Trading usually ceases a few days before the last day on which delivery can be made. 


## Hedging Strategies Using Futures



Perfect heding situations are rare for a few reasons so there is **basis risk.**

\begin{align}

Basis = S_{t} - F_{s}(\tau,T)

\end{align}

- The equation may be switched- particulary if the asset being hedged is a financial asset. If the underlying asset in the futures contract is the same as the spot price asset the basis converges to $0$.

## Cross Hedging

- If you need to hedge against price movements of an asset purchased in your portfolio and there is not a specific contract that underlying is the same you must choose a contract that is closely related to that asset. In order to determine the *hedge ratio* - the sensitivity of the asset price with respect to a one-unit change in the hedged asset. 

\begin{align}

r_{s} = \alpha + \beta_{f} +\varepsilon

\end{align}

The number of contracts used to hedge a **real asset**:

\begin{align}

N^{\star} &= \beta_{F}\frac{Q_{a}}{Q_{F}} \\

Q_{a} \ \text{and} \ Q_b \ \text{are the sizes of your positions}

\end{align}

The number of contracts used to hedge an equity portfolio using **index futures**:

\begin{align}

N^{\star} &= \beta_{F}\frac{V_{a}}{V_{F}} \\

V_{a} \ \text{and} \ V_{F} \ \text{are the mkt vals of each portfolio of assets}

\end{align}


## Example

Suppose a portfolio needs to be hedged against systematic risk in the next three months. The $V_{t=0,p} = \$5,050,000$ and $S_{t=0,A} = \$4,551.71$ is the current price of the S&P 500. The $\beta_{t=0,m} = 1.5$ and the current futures price of a E-mini S&P 500 contract expiring in March 2024 (4 months) is $\text{ESH4} = \$4,595.5$. The current date is December 1, 2023. 
Simulate the value of the portfolio at the end of February $t=3$ for different returns on the S&P 500 index. The current dividend yield on the index is $1.5\%$ per annum and risk-free rate is $4.00\%$ per annum.

\begin{align}

\mathbb{E}[V_{t,p}] &= r_{f} + \beta(r_{m}- r_{f}) \\

V_{t=0,ESH4} &= 50\cdot 4595.5 = \$229,775 \\

N^{\star} &= 1.5 \cdot \frac{\$5,050,000}{\$229,775} \approx 33

\end{align}

- Thus, approximately 33 contracts should be shorted to hedge systematic risk. 



Using Geometric Brownian Motion we can simulate the stock price at the end of the 3 months. T = $\frac{1}{4}$.

\begin{align}

\mathbb{E}[S_{t=.25}] = S_{t=0}\exp\{\sigma\sqrt{t}W(t) + (r - \frac{\sigma^{2}}{2})t\}

\end{align}



In [68]:
multiplier = 50
St = 4551.12
Beta = 1.5
Vp = 5050*1e3
dividend = 0.015
ESH4 = 4595.5
rf = 0.04
rf_3 = (rf/12)*3
e_d3  = (dividend/12)*3
vol = .3
current_index = 4551.12
T = 3/12

In [465]:
class Hedging_Simulation:
    def __init__(self,S0,sigma, RF, T, I,DIVIDEND_YIELD,BETA, FUT_MULT, FUT_PRICE, Equity_Portfolio):

        """
        S0 : The current price of the asset
        sigma: annualized volatility
        RF: risk-free rate
        T: time period of simulation
        I: Number of simulations
        DIVIDEND_YIELD: Dividend yield on index
        Beta: The sensitivity of the equity portfolio to the index

        FUT_MULT: The futures multiplier
        FUT_PRICE: The current futures price
        Equity_Portfolio: The market value of the equity portfolio
        """

        self.S0 = S0
        self.sigma = sigma
        self.RF = RF
        self.T = T
        self.I = I
        self.DIVIDEND_YIELD = DIVIDEND_YIELD
        self.BETA = BETA 
        self.FUT_MULT = FUT_MULT
        self.FUT_PRICE = FUT_PRICE
        self.Equity_Portfolio = Equity_Portfolio
        self.basis = self.FUT_PRICE-self.S0

    def simulation(self):

        """
        S0 : The current price of the asset
        sigma: annualized volatility
        RF: risk-free rate
        T: time period of simulation
        I: Number of simulations
        
        
        Returns the possible returns and prices of index level using GBM and the futures price
        The model assumes the basis at the end of 3 months for the Futures Price is $10


        """
        S0, T,sigma, I, RF = self.S0, self.T,self.sigma,self.I,self.RF

        S1 = S0*np.exp((RF-0.5*sigma**2)*T + sigma*np.sqrt(T)*npr.standard_normal(I))
        S1 = S1.reshape(len(S1),1)
        S1 = pd.DataFrame(S1, columns=['St'])
        S1.index.name = f"T: {round(T,2)}"
        S1['M_rets'] = (S1['St']/S0)-1 + self.DIVIDEND_YIELD*(T)
        S1['FUT_PRICE'] = S1['St']*(1+rf*(1/12))
        S1['R_vp'] = RF*T + self.BETA*(S1['M_rets']-RF*T)
        S1['Expected_PV'] = self.Equity_Portfolio*(1+S1['R_vp'])
        return S1
    def N_contracts(self):
        """
        Returns the Number of futures contracts to be shorted rounded to the nearest whole number
        """
        VF = self.FUT_MULT*self.FUT_PRICE
        N = self.BETA*(self.Equity_Portfolio/VF)
        return round(N,ndigits=0)
    def stock_hedge(self):

        s1 = self.simulation()
        Initial_VF = self.FUT_MULT*self.FUT_PRICE
        N = self.N_contracts()
        s1['PNL_FUT'] = -N*self.FUT_MULT*(s1['FUT_PRICE']- self.FUT_PRICE)
        s1 = s1[['M_rets','R_vp','Expected_PV','PNL_FUT']]
        s1['Total_Val'] =s1['Expected_PV'] + s1['PNL_FUT']
        s1['R_vp_hedged'] = (s1['Total_Val']/self.Equity_Portfolio)-1
        return s1
    def summary_stats(self):
        
        mean_values = self.stock_hedge().mean().to_frame('Value').T
        mean_values['M_rets'] = mean_values['M_rets'].apply(lambda x: '{:.2%}'.format(x) if pd.notnull(x) else x) # Convert to percentage
        mean_values['R_vp'] = mean_values['R_vp'].apply(lambda x: '{:.2%}'.format(x) if pd.notnull(x) else x)  # Convert to percentage
        mean_values['Expected_PV'] = mean_values['Expected_PV'].apply(lambda x: '${:,.2f}'.format(x) if pd.notnull(x) else x)

        mean_values['PNL_FUT'] = mean_values['PNL_FUT'].apply(lambda x: '${:,.2f}'.format(x) if pd.notnull(x) else x)
        mean_values['Total_Val'] = mean_values['Total_Val'].apply(lambda x: '${:,.2f}'.format(x) if pd.notnull(x) else x)
        mean_values['R_vp_hedged'] =mean_values['R_vp_hedged'].apply(lambda x: '{:.2%}'.format(x) if pd.notnull(x) else x) 

        return mean_values
    def formatted_portfolio(self):

        mean_values = self.stock_hedge()
        mean_values['M_rets'] = mean_values['M_rets'].apply(lambda x: '{:.2%}'.format(x) if pd.notnull(x) else x) # Convert to percentage
        mean_values['R_vp'] = mean_values['R_vp'].apply(lambda x: '{:.2%}'.format(x) if pd.notnull(x) else x)  # Convert to percentage
        mean_values['Expected_PV'] = mean_values['Expected_PV'].apply(lambda x: '${:,.2f}'.format(x) if pd.notnull(x) else x)

        mean_values['PNL_FUT'] = mean_values['PNL_FUT'].apply(lambda x: '${:,.2f}'.format(x) if pd.notnull(x) else x)
        mean_values['Total_Val'] = mean_values['Total_Val'].apply(lambda x: '${:,.2f}'.format(x) if pd.notnull(x) else x)
        mean_values['R_vp_hedged'] =mean_values['R_vp_hedged'].apply(lambda x: '{:.2%}'.format(x) if pd.notnull(x) else x) 

        return mean_values
    
            
sim1 = Hedging_Simulation(S0 = current_index,sigma= vol, RF = rf, I = 10000, DIVIDEND_YIELD=dividend, BETA=1.5,FUT_MULT=50,FUT_PRICE=ESH4,T = T, Equity_Portfolio=Vp)
#sim1.simulation()


In [466]:
sim1.stock_hedge()

,M_rets,R_vp,Expected_PV,PNL_FUT,Total_Val,R_vp_hedged
T: 0.25,,,,,,
0,0.105514,0.153271,5.824017e+06,-718531.503236,5.105486e+06,0.010987
1,0.014227,0.016340,5.132518e+06,-30740.806449,5.101778e+06,0.010253
2,0.058140,0.082210,5.465159e+06,-361597.313139,5.103561e+06,0.010606
3,-0.007200,-0.015799,4.970213e+06,130693.826042,5.100907e+06,0.010081
4,0.139312,0.203967,6.080035e+06,-973176.376546,5.106859e+06,0.011259
...,...,...,...,...,...,...
9995,0.104554,0.151831,5.816747e+06,-711300.489117,5.105447e+06,0.010980
9996,0.092601,0.133902,5.726206e+06,-621244.846929,5.104961e+06,0.010883
9997,-0.002473,-0.008710,5.006015e+06,95084.519824,5.101099e+06,0.010119


In [467]:
stock_hedge = sim1.stock_hedge()

In [471]:
V = sim1.summary_stats()
V

,M_rets,R_vp,Expected_PV,PNL_FUT,Total_Val,R_vp_hedged
Value,1.34%,1.51%,"$5,126,157.24","$-24,413.68","$5,101,743.56",1.02%


- As expected, the complete hedge will generate a return on our equity portfolio equal to the risk-free rate.

stock_hedge.mean()

In [441]:
stock_hedge.head(2)

,M_rets,R_vp,Expected_PV,PNL_FUT,Total_Val,R_vp_hedged
T: 0.25,,,,,,
0,-0.280533,-0.425799,2.899715e+06,2.190059e+06,5.089774e+06,0.007876
1,0.085149,0.122723,5.669751e+06,-5.651380e+05,5.104613e+06,0.010815


In [474]:
px.line(data_frame=stock_hedge, x = stock_hedge.index, y = ['Expected_PV','PNL_FUT','Total_Val'])

In [473]:
px.line(data_frame=stock_hedge,x = stock_hedge.index,y = ['M_rets','R_vp','R_vp_hedged'])

In [605]:
T = np.linspace(.5,stop = 10,num = 20,endpoint=True).reshape(20,1)
spot_rate = np.array([0.01,.015,.02,.023,0.028,.031,.0321,0.038,.041,.0442,.045,.0456,.0486,.051,.052,.0523,.0534,.0544,.0551,.0559]).reshape(20,1)
discounts = np.exp(-spot_rate*T)
cpn = (.041*100)/2
FV =100

In [606]:
Price =(sum((cpn*discounts))[0] + discounts[-1]*FV)[0]
duration = (sum(T*(cpn*discounts))   + T[-1]*FV*discounts[-1])[0]/Price
convexity = (sum(T**(2)*(cpn*discounts)) + T[-1]**(2) * FV * discounts[-1])[0]/Price
convexity

74.85473633554942

In [624]:
interest_rate_Moves = np.linspace(-.02,stop = .02,num = 1000, endpoint=True)

In [625]:
interest_rate_Moves

array([-2.00000000e-02, -1.99599600e-02, -1.99199199e-02, -1.98798799e-02,
       -1.98398398e-02, -1.97997998e-02, -1.97597598e-02, -1.97197197e-02,
       -1.96796797e-02, -1.96396396e-02, -1.95995996e-02, -1.95595596e-02,
       -1.95195195e-02, -1.94794795e-02, -1.94394394e-02, -1.93993994e-02,
       -1.93593594e-02, -1.93193193e-02, -1.92792793e-02, -1.92392392e-02,
       -1.91991992e-02, -1.91591592e-02, -1.91191191e-02, -1.90790791e-02,
       -1.90390390e-02, -1.89989990e-02, -1.89589590e-02, -1.89189189e-02,
       -1.88788789e-02, -1.88388388e-02, -1.87987988e-02, -1.87587588e-02,
       -1.87187187e-02, -1.86786787e-02, -1.86386386e-02, -1.85985986e-02,
       -1.85585586e-02, -1.85185185e-02, -1.84784785e-02, -1.84384384e-02,
       -1.83983984e-02, -1.83583584e-02, -1.83183183e-02, -1.82782783e-02,
       -1.82382382e-02, -1.81981982e-02, -1.81581582e-02, -1.81181181e-02,
       -1.80780781e-02, -1.80380380e-02, -1.79979980e-02, -1.79579580e-02,
       -1.79179179e-02, -

In [626]:
def price_bond(PRICE:float, DURATION:float, CONVEXITY:float, interest_rate_move = 1e-4):
    price = -DURATION*(interest_rate_move) + .5 *CONVEXITY*(interest_rate_move**2)
    price_wo_C = -DURATION*(interest_rate_move)

    return pd.DataFrame(np.array([price,price_wo_C]).reshape(len(interest_rate_Moves),2), columns=['With_Convexity','Without_Convexity'],index = interest_rate_Moves)
def plot_bond_shifts(PRICE:float, DURATION:float, CONVEXITY:float, interest_rate_move = 1e-4):
    DF = price_bond(PRICE = PRICE, DURATION = DURATION, CONVEXITY= CONVEXITY, interest_rate_move = interest_rate_move)
    fig = px.line(data_frame=DF, x = DF.index, y = ['With_Convexity','Without_Convexity'])
    
    #fig.add_hline(y = PRICE)
    fig.show()

In [628]:
p

,With_Convexity,Without_Convexity
-0.02000,0.176931,0.176547
-0.01996,0.176163,0.175779
-0.01992,0.175395,0.175012
-0.01988,0.174628,0.174245
-0.01984,0.173861,0.173478
...,...,...
0.01984,-0.159042,-0.159366
0.01988,-0.159690,-0.160015
0.01992,-0.160339,-0.160663
0.01996,-0.160987,-0.161312


In [627]:
p= price_bond(PRICE=Price, DURATION=duration,CONVEXITY=convexity,interest_rate_move=interest_rate_Moves)
plot_bond_shifts(PRICE=Price, DURATION=duration,CONVEXITY=convexity,interest_rate_move=interest_rate_Moves)